In [ ]:
import os
from PIL import Image, ImageDraw, ImageFont
import cv2
import numpy as np

In [ ]:
# define global constant

# image input path
g_InputFolderPath_str = r'Database\Raw_Screenshots'
g_EgoHandOutputFolderPath_str = r'Database\EgoHand_Separated_PAIs'

# crop coordinates for ego hand part, should be changed depend on your screen size
g_EgoHandCropLeftUpperX_int = 250
g_EgoHandCropLeftUpperY_int = 890
g_EgoHandCropRightLowerX_int = 1920 - g_EgoHandCropLeftUpperX_int
g_EgoHandCropRightLowerY_int = 1040

# rough area of one PAI in down part, used for thresholding rectangle detection; depend on your screen size
g_EgoHandCropMinPaiArea_int = 8000

# kernal size of gaussian filter for down part; odd integer; too small: can't ensure separation; too big: losing details 
g_EgoHandGaussKSize_int = 9

In [ ]:
def SeparationEgoHand(f_ImagePath_str, f_ImageNameNoExt_str):
    global g_EgoHandOutputFolderPath_str
    global g_EgoHandCropLeftUpperX_int
    global g_EgoHandCropLeftUpperY_int
    global g_EgoHandCropRightLowerX_int
    global g_EgoHandCropRightLowerY_int
    global g_EgoHandCropMinPaiArea_int
    global g_EgoHandGaussKSize_int
    
    # open image
    l_im_img = Image.open(f_ImagePath_str)
    
    # crop the interesting part
    l_im_img = l_im_img.crop((g_EgoHandCropLeftUpperX_int, g_EgoHandCropLeftUpperY_int, \
                              g_EgoHandCropRightLowerX_int, g_EgoHandCropRightLowerY_int))
    
    # convert into 8 bit gray scale image for further operation
    l_im_gs_img = l_im_img.convert('L')
    l_im_gs_npa = np.array(l_im_gs_img) # convert to np array for cv2
    
    # originally from game screenshot, therefor gaussian noise filter not necessary
    # still use the gaussian for better contour detection
    l_im_gs_npa = cv2.GaussianBlur(l_im_gs_npa, (g_EgoHandGaussKSize_int, g_EgoHandGaussKSize_int), 0)
    
    # detect edges with canny
    l_im_edge_npa = cv2.Canny(l_im_gs_npa, 30, 100)

    # detect counters based on edges
    # only outside edges and simple 3 directions for edges
    l_contour, _ = cv2.findContours(l_im_edge_npa, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    
    # find rectangles for each contour
    # initialize file suffix counter
    l_suffix_int = 0
    for c in l_contour:
        x, y, w, h = cv2.boundingRect(c)
        
        # in case rectangles big enough
        if w*h >= g_EgoHandCropMinPaiArea_int:
            # get the correspoding area from original image
            l_im_Pai_img = l_im_img.crop((x, y, x+w, y+h))
            # save image to output path
            l_im_Pai_img.save(g_EgoHandOutputFolderPath_str+'\\'+f_ImageNameNoExt_str+'_'+get2DigitsSuffix(l_suffix_int)+'.png')
            # release memory
            l_im_Pai_img.close()
            # increase suffix index
            l_suffix_int += 1
            # in case suffix reaches 100, break the loop -> no more 2 digits
            # since maximal 14 PAIs in hand, should never be reached!
            if l_suffix_int == 100: break

    # release memory
    l_im_img.close()
    l_im_gs_img.close()

    # return
    return

In [ ]:
def get2DigitsSuffix(f_2DigitsNumber_int):
    ans = ''
    
    if f_2DigitsNumber_int < 10: ans = '0'+str(f_2DigitsNumber_int)
    else: ans = str(f_2DigitsNumber_int)
    
    # return
    return ans

In [ ]:
# main function

# check if input folder path exists
if os.path.isdir(g_InputFolderPath_str):
    # loop over all files in input folder
    for entry in os.scandir(g_InputFolderPath_str):
        # note: no extention check
        # check whether in output folder(s) already separated picture exists
        l_SepPicNameNoExt_str = entry.name
        l_SepPicNameNoExt_str = l_SepPicNameNoExt_str[:l_SepPicNameNoExt_str.rfind('.')] # remove extention
        l_SepPicPath_str = l_SepPicNameNoExt_str + '_00.png' # change the file name
        l_SepPicPath_str = g_EgoHandOutputFolderPath_str+'\\'+l_SepPicPath_str # add output folder path
        
        if os.path.isfile(l_SepPicPath_str):
            # if already exists -> skip this picture
            pass
        else:
            # for each png, call the separation function(s)
            SeparationEgoHand(entry.path, l_SepPicNameNoExt_str)